<a href="https://colab.research.google.com/github/dhia-eddine-jedidi/Deep-Learning-Project-using-Autoencoder-CNN-LSTM/blob/main/Deep_Learning_Project_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("stefanogiannini/crema-d-video")

print("Path to dataset files:", path)

100%|██████████| 1.97G/1.97G [00:24<00:00, 85.4MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1


In [ ]:
import os
import subprocess
from concurrent.futures import ThreadPoolExecutor

input_dir = "/root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1"
output_dir = "outputMP4"
os.makedirs(output_dir, exist_ok=True)
MAX_WORKERS = 16  # parallel conversions
def convert_to_mp4(flv_path):
    base = os.path.splitext(os.path.basename(flv_path))[0]
    mp4_path = os.path.join(output_dir, f"{base}.mp4")

    # Re-encode video (H.264) + audio (AAC) → MP4
    result = subprocess.run([
        "ffmpeg", "-y", "-i", flv_path,
        "-c:v", "libx264", "-preset", "fast", "-crf", "23",
        "-c:a", "aac", "-b:a", "128k",
        mp4_path
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    if result.returncode == 0:
        print(f"[✅] Converted: {flv_path} → {mp4_path}")
        return True
    else:
        print(f"[❌] Failed: {flv_path}")
        print(result.stderr.decode('utf-8'))
        return False

def convert_all():
    flv_files = [os.path.join(input_dir, f) for f in os.listdir(input_dir) if f.endswith(".flv")]
    if not flv_files:
        print("⚠️ No .flv files found.")
        return

    print(f"🔄 Found {len(flv_files)} .flv files — starting conversion with re-encoding...")
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        list(executor.map(convert_to_mp4, flv_files))

    print("\n✅ Conversion complete!")

if __name__ == "__main__":
    convert_all()

Streaming output truncated to the last 5000 lines.
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1025_TAI_NEU_XX.flv → outputMP4/1025_TAI_NEU_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1051_ITS_SAD_XX.flv → outputMP4/1051_ITS_SAD_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1058_TSI_ANG_XX.flv → outputMP4/1058_TSI_ANG_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1058_TSI_HAP_XX.flv → outputMP4/1058_TSI_HAP_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1038_ITS_SAD_XX.flv → outputMP4/1038_ITS_SAD_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1050_MTI_ANG_XX.flv → outputMP4/1050_MTI_ANG_XX.mp4
[✅] Converted: /root/.cache/kagglehub/datasets/stefanogiannini/crema-d-video/versions/1/1056_IOM_HAP_XX.flv →

In [ ]:
# Zip the folder

import shutil

zip_name = "cremad_videos.zip"
shutil.make_archive("cremad_videos", 'zip', output_dir)
print(f"✅ Folder compressed to {zip_name}")

✅ Folder compressed to cremad_videos.zip


In [ ]:
import os

zip_path = "cremad_videos.zip"

if os.path.exists(zip_path):
    print("✅ File exists.")
    print(f"File size: {os.path.getsize(zip_path)/1e6:.2f} MB")
else:
    print("❌ File not found — check your path.")


✅ File exists.
File size: 56.62 MB


In [ ]:
import os
import shutil
from tqdm import tqdm  # optional, for a progress bar

# Folder where all .flv files currently are
source_dir = "C:/Users/Dhiaj/Downloads/cremad_videos"     # change if needed
output_dir = "C:/Users/Dhiaj/Downloads/CREMA-D-Sorted1" # will be created automatically

# Mapping short emotion codes to full names
emotion_map = {
    "ANG": "Anger",
    "DIS": "Disgust",
    "FEA": "Fear",
    "HAP": "Happy",
    "NEU": "Neutral",
    "SAD": "Sad"
}

# Create target folders
for emotion_name in emotion_map.values():
    os.makedirs(os.path.join(output_dir, emotion_name), exist_ok=True)

# Loop over all videos
for filename in tqdm(os.listdir(source_dir)):
    if filename.endswith(".mp4"):
        parts = filename.replace(".mp4", "").split("_")
        if len(parts) == 4:
            emotion_code = parts[2]
            emotion_full = emotion_map.get(emotion_code)
            if emotion_full:
                src = os.path.join(source_dir, filename)
                dst = os.path.join(output_dir, emotion_full, filename)
                shutil.copy2(src, dst)  # or use shutil.move() if you want to move
        else:
            print(f"⚠️ Skipping malformed filename: {filename}")

print("✅ All videos rearranged by emotion!")


100%|█████████████████████████████████████████████████████████████████████████████| 7442/7442 [00:08<00:00, 846.05it/s]


✅ All videos rearranged by emotion!


In [ ]:
import os
import cv2
from tqdm import tqdm

# Input: sorted emotion folders with videos
videos_root = "C:/Users/Dhiaj/Downloads/CREMA-D-Sorted1"
# Output: matching folder structure for frames
frames_root = "C:/Users/Dhiaj/Downloads/Df_Frames"
os.makedirs(frames_root, exist_ok=True)

for emotion in os.listdir(videos_root):
    emotion_dir = os.path.join(videos_root, emotion)
    if not os.path.isdir(emotion_dir):
        continue

    print(f"\n🎞️ Extracting frames for emotion: {emotion}")
    os.makedirs(os.path.join(frames_root, emotion), exist_ok=True)

    # Process each video file
    for video_file in tqdm(os.listdir(emotion_dir)):
        if not video_file.endswith((".mp4", ".flv")):
            continue

        video_path = os.path.join(emotion_dir, video_file)
        base_name = os.path.splitext(video_file)[0]
        out_dir = os.path.join(frames_root, emotion, base_name)
        os.makedirs(out_dir, exist_ok=True)

        vidcap = cv2.VideoCapture(video_path)
        if not vidcap.isOpened():
            print(f"⚠️ Could not open {video_path}")
            continue

        success, frame = vidcap.read()
        frame_idx = 0
        while success:
            frame_name = f"frame_{frame_idx:04d}.jpg"
            cv2.imwrite(os.path.join(out_dir, frame_name), frame)
            frame_idx += 1
            success, frame = vidcap.read()

        vidcap.release()
        print(f"✅ {video_file}: {frame_idx} frames extracted → {out_dir}")

print("\n✅ All videos processed and frames saved by emotion!")



🎞️ Extracting frames for emotion: Anger


  0%|                                                                                 | 1/1271 [00:00<03:55,  5.40it/s]

✅ 1001_DFA_ANG_XX.mp4: 68 frames extracted → C:/Users/Dhiaj/Downloads/Df_Frames\Anger\1001_DFA_ANG_XX


  0%|▏                                                                                | 3/1271 [00:00<02:26,  8.64it/s]

✅ 1001_IEO_ANG_HI.mp4: 59 frames extracted → C:/Users/Dhiaj/Downloads/Df_Frames\Anger\1001_IEO_ANG_HI
✅ 1001_IEO_ANG_LO.mp4: 63 frames extracted → C:/Users/Dhiaj/Downloads/Df_Frames\Anger\1001_IEO_ANG_LO


...

✅ 1091_TAI_SAD_XX.mp4: 66 frames extracted → C:/Users/Dhiaj/Downloads/Df_Frames\Sad\1091_TAI_SAD_XX


100%|██████████████████████████████████████████████████████████████████████████████| 1271/1271 [03:28<00:00,  4.29it/s]

✅ 1091_TIE_SAD_XX.mp4: 101 frames extracted → C:/Users/Dhiaj/Downloads/Df_Frames\Sad\1091_TIE_SAD_XX
✅ 1091_TSI_SAD_XX.mp4: 81 frames extracted → C:/Users/Dhiaj/Downloads/Df_Frames\Sad\1091_TSI_SAD_XX
✅ 1091_WSI_SAD_XX.mp4: 76 frames extracted → C:/Users/Dhiaj/Downloads/Df_Frames\Sad\1091_WSI_SAD_XX


100%|██████████████████████████████████████████████████████████████████████████████| 1271/1271 [03:28<00:00,  6.10it/s]


✅ All videos processed and frames saved by emotion!


In [ ]:
import os
import shutil
from tqdm import tqdm

# Root folder with current structure
frames_root = "C:/Users/Dhiaj/Downloads/Df_Frames"  # contains subfolders per emotion, each with subfolders per video

# Loop over each emotion folder
for emotion in os.listdir(frames_root):
    emotion_dir = os.path.join(frames_root, emotion)
    if not os.path.isdir(emotion_dir):
        continue

    print(f"\n🎞️ Merging frames for emotion: {emotion}")
    count = 0

    # Loop through subfolders (each video)
    for video_folder in tqdm(os.listdir(emotion_dir)):
        video_path = os.path.join(emotion_dir, video_folder)
        if not os.path.isdir(video_path):
            continue

        # Move each frame into the parent emotion folder
        for frame_file in os.listdir(video_path):
            if not frame_file.lower().endswith((".jpg", ".png")):
                continue

            src = os.path.join(video_path, frame_file)
            # Rename frame to include video folder name (so no duplicates)
            new_name = f"{video_folder}_{frame_file}"
            dst = os.path.join(emotion_dir, new_name)

            shutil.move(src, dst)
            count += 1

        # Remove now-empty subfolder
        os.rmdir(video_path)

    print(f"✅ {emotion}: {count} frames merged into one folder")

print("\n✅ All emotion folders flattened successfully!")



🎞️ Merging frames for emotion: Anger


100%|██████████████████████████████████████████████████████████████████████████████| 1271/1271 [01:37<00:00, 13.03it/s]


✅ Anger: 99983 frames merged into one folder

🎞️ Merging frames for emotion: Disgust


100%|██████████████████████████████████████████████████████████████████████████████| 1271/1271 [01:52<00:00, 11.29it/s]


✅ Disgust: 109272 frames merged into one folder

🎞️ Merging frames for emotion: Fear


...

✅ Happy: 90665 frames merged into one folder

🎞️ Merging frames for emotion: Neutral


100%|██████████████████████████████████████████████████████████████████████████████| 1087/1087 [01:32<00:00, 11.76it/s]


✅ Neutral: 80309 frames merged into one folder

🎞️ Merging frames for emotion: Sad


100%|██████████████████████████████████████████████████████████████████████████████| 1271/1271 [01:50<00:00, 11.51it/s]

✅ Sad: 100530 frames merged into one folder

✅ All emotion folders flattened successfully!
